In [1]:
from clifford.g4 import *
import numpy as np
from clifford import MultiVector, MVArray
import numbers

/work/anaconda/lib/python3.7/site-packages/numba/compiler.py:602: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see http://numba.pydata.org/numba-doc/latest/user/parallel.html#diagnostics for help.

File "../../../work/anaconda/lib/python3.7/site-packages/clifford/__init__.py", line 83:
    @numba.njit(parallel=NUMBA_PARALLEL)
    def play_games():
    ^

  self.func_ir.loc))


In [2]:
# Using a normal 4d space
print(e1*e1)
print(e2*e2)
print(e3*e3)
print(e4*e4)

1.0
1.0
1.0
1.0


In [3]:
class HybridMV:
    def __init__(self, internal_array=None):
        if internal_array is None:
            self.internal_array = MVArray([0*e1, 0*e1])
        else:
            self.internal_array = MVArray(internal_array)
            
    def __add__(self, other):
        if issubclass(other.__class__, HybridMV):
            return HybridMV(self.internal_array + other.internal_array)
        else:
            new_mv = HybridMV(self.internal_array)
            new_mv.internal_array[0] += other
            return new_mv
    
    def __rmul__(self, other):
        if issubclass(other.__class__, numbers.Number):
            return HybridMV(other*self.internal_array)
    
    def __mul__(self, other):
        if issubclass(other.__class__, numbers.Number):
            return HybridMV(self.internal_array*other)
        if issubclass(other.__class__, MultiVector):
            return HybridMV(self.internal_array*other)
        elif issubclass(other.__class__, HybridMV):
            newmv = HybridMV(self.internal_array*other.internal_array[0])
            newmv.internal_array[1] += self.internal_array[0]*other.internal_array[1]
            return newmv
        
    def __xor__(self, other):
        if issubclass(other.__class__, MultiVector):
            return HybridMV(self.internal_array^other)
        elif issubclass(other.__class__, HybridMV):
            newmv = HybridMV(self.internal_array^other.internal_array[0])
            newmv.internal_array[1] += self.internal_array[0]^other.internal_array[1]
            return newmv
        
    def __pow__(self, other):
        new = HybridMV([1])
        for i in range(other):
            new = new*self
        return new
    
    def __or__(self, other):
        if issubclass(other.__class__, MultiVector):
            return HybridMV(self.internal_array|other)
        elif issubclass(other.__class__, HybridMV):
            newmv = HybridMV(self.internal_array|other.internal_array[0])
            newmv.internal_array[1] += self.internal_array[0]|other.internal_array[1]
            return newmv
        
    __ror__ = __or__
    
    def __str__(self):
        total_string = ''
        for i in range(len(self.internal_array)):
            if i > 0:
                total_string +='d' + str(i) + '*[' + str(self.internal_array[i]) + ']'
            else:
                total_string += str(self.internal_array[i])
            if i < len(self.internal_array) -1:
                total_string+= ' + '
        return total_string
    
    def __repr__(self):
        return str(self)
    
    @property
    def minterm(self):
        for i in range(len(self.internal_array)):
            if self.internal_array[i] != 0*e1:
                return i
    
    def trim(self, order=1):
        """
        Removes anything above the order specified
        """
        return HybridMV(self.internal_array[0:order+1])
    
    def homo(self):
        return HybridMV(self.internal_array/(self.internal_array[0][4]))

    def __invert__(self):
        return HybridMV(~self.internal_array)
    
    def to_point(self):
        return self.homo().internal_array[1] + e4
    
    @staticmethod
    def from_point(X):
        newmv = HybridMV()
        newmv.internal_array[0][4] = X[4]
        newmv.internal_array[1].value[1:4] = X.value[1:4]
        return newmv
    
    def apply(self, other):
        return (R*HybridMV.from_point(other)*~R).to_point()
    

In [4]:
# Create the dual unit
d1 = HybridMV([0*e1, 0*e1 + 1.0])
d1

0 + d1*[1.0]

In [5]:
d1*((d1 + 2*d1*e12) + e1234)

0 + d1*[(1.0^e1234)]

In [6]:
I4 = e1234

def up(x):
    return x + e4

def homo(x):
    return x/((x|e4)[0])

def down(x):
    return homo(x)(e123)

def apply(X,R):
    return R.apply(X)

def meet(X, Y):
    return ((X*I4)^(Y*I4))*I4

def random_euc_point():
    return random.randn()*e1 + random.randn()*e2 + random.randn()*e3

def random_point():
    return up(random_euc_point())

# Lets have a go with intersecting a plane and a line

In [7]:
# Create some points
A = up(1.0*e1)
B = up(1.0*e2)
C = up(2.0*e1 - 3.0*e2 + 1.0*e3)
print(A)
print(B)
print(C)

(1.0^e1) + (1.0^e4)
(1.0^e2) + (1.0^e4)
(2.0^e1) - (3.0^e2) + (1.0^e3) + (1.0^e4)


In [8]:
# Wedge the points to give a line and a plane
L1 = A^C
P2 = B^C^up(0*e1)
print(L1)
print(P2)

-(3.0^e12) + (1.0^e13) - (1.0^e14) + (3.0^e24) - (1.0^e34)
-(2.0^e124) + (1.0^e234)


In [9]:
# Perform the meet as usual with the dual and wedge
meet(L1,P2)

(2.0^e1) - (3.0^e2) + (1.0^e3) + (1.0^e4)

In [10]:
# Homogenise to get the point at the intersection
meet(L1,P2)

(2.0^e1) - (3.0^e2) + (1.0^e3) + (1.0^e4)

In [11]:
# Call down to convert back to a normal euc point
down(meet(L1,P2))

(2.0^e1) - (3.0^e2) + (1.0^e3)

In [12]:
%%timeit
A*C

2.86 µs ± 89 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [13]:
%%timeit
d1*C

33.4 µs ± 2.39 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [14]:
d1*C

0 + d1*[(2.0^e1) - (3.0^e2) + (1.0^e3) + (1.0^e4)]

# Translators

In [20]:
def translation_rotor(t):
    return 0.5*(d1*t)*e4 + 1

In [21]:
R = translation_rotor(10*e1+5*e3)
print(R)
print(R*~R)

1.0 + d1*[(5.0^e14) + (2.5^e34)]
1.0 + d1*[0]


In [22]:
R*C*~R

(2.0^e1) - (3.0^e2) + (1.0^e3) + (1.0^e4) + d1*[(10.0^e1) + (5.0^e3) - (25.0^e4)]

In [23]:
C

(2.0^e1) - (3.0^e2) + (1.0^e3) + (1.0^e4)

In [24]:
apply(C,R)

(12.0^e1) - (3.0^e2) + (6.0^e3) + (1.0^e4)

In [25]:
%%timeit
apply(C,R)

144 µs ± 1.81 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [26]:
%%prun -s cumulative
for i in range(10000):
    apply(C,R)

In [27]:
apply(C,R)

(12.0^e1) - (3.0^e2) + (6.0^e3) + (1.0^e4)

# Rotation rotors are the same as in 3D

In [28]:
def rotation_rotor(m,n,theta):
    B = (m*n).normal()
    rotor = np.cos(theta / 2) - B * np.sin(theta / 2)
    return rotor

In [29]:
R = rotation_rotor(e1,e2,np.pi/4)
print(R)
print(R*~R)

0.92388 - (0.38268^e12)
1.0


In [30]:
print(R*(up(e1)*~R))
print(down(R*(up(e1)*~R)))
print(R*e1*~R)
print(down((R*C*~R)))
print(down((R*(C*I4)*~R)*I4))

(0.70711^e1) + (0.70711^e2) + (1.0^e4)
(0.70711^e1) + (0.70711^e2)
(0.70711^e1) + (0.70711^e2)
(3.53553^e1) - (0.70711^e2) + (1.0^e3)
(3.53553^e1) - (0.70711^e2) + (1.0^e3)
